In [18]:
import os
import rasterio
import cv2 as cv
import sys
import pandas as pd
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import shutil

sys.path.insert(1, "../")
from utils import *
import shlex
from subprocess import run

In [ ]:
%%bash

module use /g/data/v10/public/modules/modulefiles
module use /g/data/v10/private/modules/modulefiles
module load ard-pipeline/20240607-1302-s2

export LD_LIBRARY_PATH=/g/data/v10/projects/geo_assess/tool/lib_v0.25:/g/data/v10/projects/PinkMatter/ExtraLibs/Jasper:$LD_LIBRARY_PATH
export GDAL_DATA=/g/data/v10/projects/geo_assess/tool/lib_v0.25/gdal_data
export GEOTIFF_CSV=/g/data/v10/projects/geo_assess/tool/lib_v0.25/epsg_csv

In [192]:
Tile = "T42DWK"
gv_temp_dir = (
    f"/g/data/yp75/projects/ar-coreg/notebooks/data/outputs/GVerify/{Tile}/temp"
)
gv_outp_dir = (
    f"/g/data/yp75/projects/ar-coreg/notebooks/data/outputs/GVerify/{Tile}/out"
)
gv_logs_dir = (
    f"/g/data/yp75/projects/ar-coreg/notebooks/data/outputs/GVerify/{Tile}/log"
)

res_file = f"../data/outputs/GVerify/{Tile}/out/image-gverify.res"
ref_scene = f"/g/data/yp75/projects/ar-coreg/notebooks/data/outputs/GVerify/inputs/{Tile}/ref.tif"
tgt_scene = f"/g/data/yp75/projects/ar-coreg/notebooks/data/outputs/GVerify/inputs/{Tile}/tgt.tif"
# ref_scene = "/g/data/yp75/projects/ar-coreg/notebooks/data/outputs/AROSICS/RTC_Series/S1A__IW___A_20230312T083311_VV_gamma0-rtc.tif"
# tgt_scene = "/g/data/yp75/projects/ar-coreg/notebooks/data/outputs/AROSICS/RTC_Series/S1A__IW___A_20230417T083309_HH_gamma0-rtc.tif"

shutil.rmtree(gv_temp_dir, ignore_errors=True)
shutil.rmtree(gv_outp_dir, ignore_errors=True)
shutil.rmtree(gv_logs_dir, ignore_errors=True)
os.makedirs(gv_temp_dir, exist_ok=True)
os.makedirs(gv_outp_dir, exist_ok=True)
os.makedirs(gv_logs_dir, exist_ok=True)

os.makedirs(f"../data/outputs/GVerify/inputs/{Tile}", exist_ok=True)

chip_method = "EVEN"  # (SOBEL, EVEN, HARRIS, FIXED_LOCATION)
pyramid_level = 1
null_value = 0
corr_coef = 0.75
resampling = "BI"
chip_size = 50
grid_size = chip_size * 2
use_phase_corr = "false"
if chip_method == "FIXED_LOCATION":
    t_file_cmd = " -t_file /g/data/up71/projects/gverify_testing/1_can_optical_imagery_as_a_reference_image/ponts_clipped.txt"
else:
    t_file_cmd = ""

In [193]:
img = rasterio.open(
    "../data/outputs/AROSICS/L1C_AN_Series_T42DWK_DS/T42DWK_20230210T041729_TCI.jp2"
)
profile = img.profile
profile["driver"] = "GTiff"
data = img.read()
with rasterio.open(
    f"../data/outputs/GVerify/inputs/{Tile}/ref.tif", "w", **profile
) as ds:
    for i in range(0, profile["count"]):
        ds.write(data[i], i + 1)

In [194]:
img = rasterio.open(
    "../data/outputs/AROSICS/L1C_AN_Series_T42DWK_DS/T42DWK_20230312T041729_TCI.jp2"
)
profile = img.profile
profile["driver"] = "GTiff"
data = img.read()
with rasterio.open(
    f"../data/outputs/GVerify/inputs/{Tile}/tgt.tif", "w", **profile
) as ds:
    for i in range(0, profile["count"]):
        ds.write(data[i], i + 1)

In [ ]:
gv_cmd = (
    f"/g/data/v10/projects/geo_assess/tool/image-gverify_v0.25c "
    + f"-b {ref_scene} -m {tgt_scene} -w {gv_temp_dir} -o {gv_outp_dir} -l {gv_logs_dir} -nv 0 -p {pyramid_level} "
    + f"-n 8 -c {corr_coef} -r {resampling} -cs {chip_size} -st true -su true -g {grid_size} "
    + f"-t {chip_method}{t_file_cmd}"
)  # -usePhaseCorrelation {use_phase_corr}"
print(gv_cmd)

In [ ]:
show(
    rasterio.open(f"../data/outputs/GVerify/{Tile}/out/image-gverify_result.tif").read()
)

In [197]:
with open(res_file, "r") as f:
    for i, line in enumerate(f):
        if i == 3:
            shift_x = float(line.strip().split(" ")[3])
        elif i == 4:
            shift_y = float(line.strip().split(" ")[3])
        else:
            continue

In [ ]:
ref_img = cv.cvtColor(flip_img(rasterio.open(ref_scene).read()), cv.COLOR_BGR2GRAY)
tgt_img = cv.cvtColor(flip_img(rasterio.open(tgt_scene).read()), cv.COLOR_BGR2GRAY)
# ref_img = flip_img(rasterio.open(ref_scene).read())[:, :, 0].astype("uint8")
# tgt_img = flip_img(rasterio.open(tgt_scene).read())[:, :, 0].astype("uint8")
tgt_aligned = warp_affine_dataset(
    tgt_img, translation_x=shift_x, translation_y=shift_y
).astype("uint8")
tgt_aligned[tgt_aligned <= np.min(tgt_img)] = np.min(tgt_img)
_, axes = plt.subplots(1, 3, figsize=(15, 10))
axes[0].imshow(ref_img)
axes[0].set_xlabel("Reference")
axes[1].imshow(tgt_img)
axes[1].set_xlabel(
    f"Target, ssim:{np.round(ssim(ref_img, tgt_img, win_size=3), 3)}, mse:{np.round(mse(ref_img, tgt_img), 3)}"
)
axes[2].imshow(tgt_aligned)
axes[2].set_xlabel(
    f"Registered, ssim:{np.round(ssim(ref_img, tgt_aligned, win_size=3), 3)}, mse:{np.round(mse(ref_img, tgt_aligned), 3)}"
)
plt.tight_layout()

##### S1 SLC vs S2 L1C 

In [251]:
Tile = "SLC_L1C"
gv_temp_dir = (
    f"/g/data/yp75/projects/ar-coreg/notebooks/data/outputs/GVerify/{Tile}/temp"
)
gv_outp_dir = (
    f"/g/data/yp75/projects/ar-coreg/notebooks/data/outputs/GVerify/{Tile}/out"
)
gv_logs_dir = (
    f"/g/data/yp75/projects/ar-coreg/notebooks/data/outputs/GVerify/{Tile}/log"
)

res_file = f"../data/outputs/GVerify/{Tile}/out/image-gverify.res"
ref_scene = f"/g/data/yp75/projects/ar-coreg/notebooks/data/outputs/GVerify/inputs/{Tile}/ref.tif"
tgt_scene = (
    "/g/data/yp75/projects/ar-coreg/notebooks/data/outputs/OpenCV/L1C_SLC_AU/tgt_ds.tif"
)

shutil.rmtree(gv_temp_dir, ignore_errors=True)
shutil.rmtree(gv_outp_dir, ignore_errors=True)
shutil.rmtree(gv_logs_dir, ignore_errors=True)
os.makedirs(gv_temp_dir, exist_ok=True)
os.makedirs(gv_outp_dir, exist_ok=True)
os.makedirs(gv_logs_dir, exist_ok=True)

os.makedirs(f"../data/outputs/GVerify/inputs/{Tile}", exist_ok=True)

chip_method = "EVEN"  # (SOBEL, EVEN, HARRIS, FIXED_LOCATION)
pyramid_level = 1
null_value = 0
corr_coef = 0.75
resampling = "BI"
chip_size = 15
grid_size = chip_size * 2
use_phase_corr = "false"
if chip_method == "FIXED_LOCATION":
    t_file_cmd = " -t_file /g/data/up71/projects/gverify_testing/1_can_optical_imagery_as_a_reference_image/ponts_clipped.txt"
else:
    t_file_cmd = ""

In [219]:
img = rasterio.open("../data/outputs/OpenCV/L1C_SLC_AU/ref_ds.tif")
profile = img.profile
profile["driver"] = "GTiff"
data = img.read()
with rasterio.open(
    f"../data/outputs/GVerify/inputs/{Tile}/ref.tif", "w", **profile
) as ds:
    for i in range(0, profile["count"]):
        ds.write(data[i], i + 1)

In [ ]:
gv_cmd = (
    f"/g/data/v10/projects/geo_assess/tool/image-gverify_v0.25c "
    + f"-b {ref_scene} -m {tgt_scene} -w {gv_temp_dir} -o {gv_outp_dir} -l {gv_logs_dir} -nv 0 -p {pyramid_level} "
    + f"-n 8 -c {corr_coef} -r {resampling} -cs {chip_size} -st true -su true -g {grid_size} "
    + f"-t {chip_method}{t_file_cmd}"
)  # -usePhaseCorrelation {use_phase_corr}"
print(gv_cmd)

In [ ]:
with open(res_file, "r") as f:
    for i, line in enumerate(f):
        if i == 3:
            shift_x = float(line.strip().split(" ")[3])
        elif i == 4:
            shift_y = float(line.strip().split(" ")[3])
        else:
            continue

print(shift_x, shift_y)

In [ ]:
coords, warps = find_overlap(ref_scene, tgt_scene, True, True)
ref_warp = cv.cvtColor(warps[2], cv.COLOR_BGR2GRAY)
tgt_warp = cv.cvtColor(warps[3], cv.COLOR_BGR2GRAY)
tgt_shifted = warp_affine_dataset(
    tgt_warp, translation_x=shift_x, translation_y=shift_y
)

In [ ]:
_, axes = plt.subplots(1, 2, figsize=(15, 10))
axes[0].imshow(simple_mosaic([ref_warp, tgt_warp]), cmap="gray")
axes[0].set_xlabel(
    f"Original, ssim:{np.round(ssim(ref_warp, tgt_warp), 3)}, mse:{np.round(mse(ref_warp, tgt_warp), 3)}"
)
axes[1].imshow(simple_mosaic([ref_warp, tgt_shifted]), cmap="gray")
axes[1].set_xlabel(
    f"Registered, ssim:{np.round(ssim(ref_warp, tgt_shifted), 3)}, mse:{np.round(mse(ref_warp, tgt_shifted), 3)}"
)

In [ ]:
# number_of_iterations = 10
# termination_eps = 1e-5
# of_params: dict = dict(
#     # params for ShiTomasi corner detection
#     feature_params=dict(
#         maxCorners=1000,
#         qualityLevel=0.3,
#         minDistance=7,
#         blockSize=7,
#     ),
#     # Parameters for lucas kanade optical flow
#     lk_params=dict(
#         winSize=(15, 15),
#         maxLevel=1,
#     ),
# )

# coords, warps = find_overlap(ref_scene, tgt_scene, True, True)
# ref_warp = cv.cvtColor(warps[2], cv.COLOR_BGR2GRAY)
# tgt_warp = cv.cvtColor(warps[3], cv.COLOR_BGR2GRAY)
# # ref_warp = warps[2]
# # tgt_warp = warps[3]
# tgt_aligned_list, shifts = co_register(
#     ref_warp,
#     tgt_warp,
#     filtering_mode="of",
#     output_path="../data/outputs/temp",
#     of_params=of_params,
#     number_of_iterations=number_of_iterations,
#     termination_eps=termination_eps,
#     enhanced_shift_method="corr",
#     remove_outlilers=True,
#     corr_thresh=0.1,
# )